# Noções de IA - Trabalho 3 - Rede Convolucional e Transfer Learning
##### Integrantes:
- Lucas Corrêa Boaventura           - 211038262
- Alexandre Castanho Sampaio Rocha  - 211027536

### Bibliotecas

In [13]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers, models
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

### Banco de Dados de Flores


In [14]:
(train_ds, test_ds) = tfds.load(
    'tf_flowers',
    split=['train[:70%]', 'train[70%:]'],
    as_supervised=True
)

train_ds, train_labels = tuple(zip(*train_ds))
test_ds, test_labels = tuple(zip(*test_ds))
L = [tf.image.resize(tf.convert_to_tensor(x), (150, 150)) for x in train_ds]
train_ds = tf.convert_to_tensor(L)
L = [tf.image.resize(tf.convert_to_tensor(x), (150, 150)) for x in test_ds]
test_ds = tf.convert_to_tensor(L)

In [15]:
## Resizing images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))
## Transforming labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)
print(train_ds.shape)
print(test_ds.shape)

(2569, 150, 150, 3)
(1101, 150, 150, 3)


### Perpectron Multi Camada

In [16]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(128, activation='relu')
dense_layer_2 = layers.Dense(64, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')

model = models.Sequential([
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


## Treinamento com parada prematura

In [17]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)
model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=128, callbacks=[es])

Epoch 1/50
17/17 [==============================] - 2s 104ms/step - loss: 2387.5881 - accuracy: 0.2336 - val_loss: 860.3568 - val_accuracy: 0.2724
Epoch 2/50
17/17 [==============================] - 2s 98ms/step - loss: 511.2271 - accuracy: 0.3280 - val_loss: 757.0029 - val_accuracy: 0.1984
Epoch 3/50
17/17 [==============================] - 2s 95ms/step - loss: 600.7842 - accuracy: 0.3285 - val_loss: 700.5334 - val_accuracy: 0.3093
Epoch 4/50
17/17 [==============================] - 2s 96ms/step - loss: 336.9628 - accuracy: 0.3776 - val_loss: 377.2205 - val_accuracy: 0.3268
Epoch 5/50
17/17 [==============================] - 2s 100ms/step - loss: 325.1712 - accuracy: 0.3796 - val_loss: 389.8188 - val_accuracy: 0.3268
Epoch 6/50
17/17 [==============================] - 2s 95ms/step - loss: 211.1992 - accuracy: 0.4443 - val_loss: 236.5327 - val_accuracy: 0.3385
Epoch 7/50
17/17 [==============================] - 2s 96ms/step - loss: 302.6875 - accuracy: 0.3815 - val_loss: 478.1901 - val

## Avaliação do modelo

In [18]:
# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_ds, test_labels)
# Print the accuracy
print('Accuracy:', accuracy)

35/35 [==============================] - 0s 8ms/step - loss: 148.7430 - accuracy: 0.3924
Accuracy: 0.3923705816268921


In [19]:
## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False ## Not trainable weights
## Preprocessing input
train_ds = preprocess_input(train_ds)
test_ds = preprocess_input(test_ds)
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [20]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(128, activation='relu')
dense_layer_2 = layers.Dense(64, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')

model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [21]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)
model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=128, callbacks=[es])

Epoch 1/50
17/17 [==============================] - 79s 5s/step - loss: 4.3906 - accuracy: 0.5908 - val_loss: 1.2268 - val_accuracy: 0.7782
Epoch 2/50
17/17 [==============================] - 78s 5s/step - loss: 0.6096 - accuracy: 0.8569 - val_loss: 1.0786 - val_accuracy: 0.7957
Epoch 3/50
17/17 [==============================] - 77s 5s/step - loss: 0.1915 - accuracy: 0.9465 - val_loss: 1.2473 - val_accuracy: 0.7879
Epoch 4/50
17/17 [==============================] - 77s 5s/step - loss: 0.0584 - accuracy: 0.9835 - val_loss: 1.0544 - val_accuracy: 0.7996
Epoch 5/50
17/17 [==============================] - 77s 5s/step - loss: 0.0175 - accuracy: 0.9961 - val_loss: 1.0714 - val_accuracy: 0.8054
Epoch 6/50
17/17 [==============================] - 77s 5s/step - loss: 0.0066 - accuracy: 1.0000 - val_loss: 1.0387 - val_accuracy: 0.8093
Epoch 7/50
17/17 [==============================] - 78s 5s/step - loss: 0.0041 - accuracy: 1.0000 - val_loss: 1.0517 - val_accuracy: 0.8035
Epoch 8/50
17/17 [==

In [22]:
# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_ds, test_labels)
# Print the accuracy
print('Accuracy:', accuracy)

35/35 [==============================] - 33s 951ms/step - loss: 1.1618 - accuracy: 0.8247
Accuracy: 0.8247048258781433


In [23]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(128, activation='relu')
dense_layer_2 = layers.Dense(64, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')

model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    tf.keras.layers.Dropout(0.5),
    dense_layer_2,
    tf.keras.layers.Dropout(0.5),
    prediction_layer
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [26]:
model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=128, callbacks=[es])

Epoch 1/50


17/17 [==============================] - 80s 5s/step - loss: 1.0889 - accuracy: 0.6127 - val_loss: 0.8175 - val_accuracy: 0.7393
Epoch 2/50
17/17 [==============================] - 84s 5s/step - loss: 1.0893 - accuracy: 0.6389 - val_loss: 0.8616 - val_accuracy: 0.7374
Epoch 3/50
17/17 [==============================] - 82s 5s/step - loss: 1.0476 - accuracy: 0.6224 - val_loss: 0.8180 - val_accuracy: 0.7354
Epoch 4/50
17/17 [==============================] - 80s 5s/step - loss: 0.8922 - accuracy: 0.6647 - val_loss: 0.8116 - val_accuracy: 0.7412
Epoch 5/50
17/17 [==============================] - 79s 5s/step - loss: 0.9163 - accuracy: 0.6584 - val_loss: 0.8380 - val_accuracy: 0.7471
Epoch 6/50
17/17 [==============================] - 73s 4s/step - loss: 0.7960 - accuracy: 0.6813 - val_loss: 0.8525 - val_accuracy: 0.7451
Epoch 7/50
17/17 [==============================] - 73s 4s/step - loss: 0.8125 - accuracy: 0.6725 - val_loss: 0.9113 - val_accuracy: 0.7451
Epoch 8/50
17/17 [=============

In [27]:
# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_ds, test_labels)
# Print the accuracy
print('Accuracy:', accuracy)

35/35 [==============================] - 32s 907ms/step - loss: 0.8293 - accuracy: 0.8029
Accuracy: 0.8029064536094666


In [28]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(128, activation='relu', kernel_regularizer='l1')
dense_layer_2 = layers.Dense(64, activation='relu', kernel_regularizer='l1')
prediction_layer = layers.Dense(5, activation='softmax')

model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [29]:
model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=128, callbacks=[es])

Epoch 1/50
17/17 [==============================] - 75s 4s/step - loss: 123.1711 - accuracy: 0.6102 - val_loss: 88.1856 - val_accuracy: 0.7237
Epoch 2/50
17/17 [==============================] - 74s 4s/step - loss: 68.2404 - accuracy: 0.8754 - val_loss: 49.6919 - val_accuracy: 0.7918
Epoch 3/50
17/17 [==============================] - 74s 4s/step - loss: 38.9282 - accuracy: 0.9416 - val_loss: 29.3293 - val_accuracy: 0.7879
Epoch 4/50
17/17 [==============================] - 74s 4s/step - loss: 23.5292 - accuracy: 0.9698 - val_loss: 18.9673 - val_accuracy: 0.7957
Epoch 5/50
17/17 [==============================] - 74s 4s/step - loss: 16.1664 - accuracy: 0.9401 - val_loss: 14.0783 - val_accuracy: 0.7724
Epoch 6/50
17/17 [==============================] - 74s 4s/step - loss: 12.1763 - accuracy: 0.9479 - val_loss: 11.2982 - val_accuracy: 0.7685
Epoch 7/50
17/17 [==============================] - 74s 4s/step - loss: 10.1161 - accuracy: 0.9255 - val_loss: 9.9774 - val_accuracy: 0.7899
Epoch 

In [30]:
# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_ds, test_labels)
# Print the accuracy
print('Accuracy:', accuracy)

35/35 [==============================] - 31s 891ms/step - loss: 4.9349 - accuracy: 0.8011
Accuracy: 0.8010899424552917


In [31]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(128, activation='relu', kernel_regularizer='l2')
dense_layer_2 = layers.Dense(64, activation='relu', kernel_regularizer='l2')
prediction_layer = layers.Dense(5, activation='softmax')

model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [32]:
model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=128, callbacks=[es])

Epoch 1/50
17/17 [==============================] - 75s 4s/step - loss: 6.9819 - accuracy: 0.6282 - val_loss: 4.8500 - val_accuracy: 0.7490
Epoch 2/50
17/17 [==============================] - 72s 4s/step - loss: 3.6857 - accuracy: 0.9046 - val_loss: 4.4791 - val_accuracy: 0.7704
Epoch 3/50
17/17 [==============================] - 72s 4s/step - loss: 3.2573 - accuracy: 0.9684 - val_loss: 4.2338 - val_accuracy: 0.7938
Epoch 4/50
17/17 [==============================] - 72s 4s/step - loss: 3.0125 - accuracy: 0.9927 - val_loss: 4.0574 - val_accuracy: 0.8016
Epoch 5/50
17/17 [==============================] - 72s 4s/step - loss: 2.8115 - accuracy: 0.9985 - val_loss: 3.8922 - val_accuracy: 0.7996
Epoch 6/50
17/17 [==============================] - 72s 4s/step - loss: 2.6253 - accuracy: 1.0000 - val_loss: 3.6883 - val_accuracy: 0.8016
Epoch 7/50
17/17 [==============================] - 72s 4s/step - loss: 2.4531 - accuracy: 1.0000 - val_loss: 3.5112 - val_accuracy: 0.8093
Epoch 8/50
17/17 [==

In [33]:
# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_ds, test_labels)
# Print the accuracy
print('Accuracy:', accuracy)

35/35 [==============================] - 31s 888ms/step - loss: 3.6716 - accuracy: 0.8084
Accuracy: 0.8083560466766357


In [34]:
## Loading resnet model
base_model = tf.keras.applications.resnet.ResNet152(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False ## Not trainable weights
## Preprocessing input
train_ds = tf.keras.applications.resnet.preprocess_input(train_ds)
test_ds = tf.keras.applications.resnet.preprocess_input(test_ds)
base_model.summary()

Model: "resnet152"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 156, 156, 3)          0         ['input_3[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 75, 75, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 75, 75, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                      

In [35]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(128, activation='relu')
dense_layer_2 = layers.Dense(64, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')

model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [36]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)
model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=128, callbacks=[es])

Epoch 1/50
17/17 [==============================] - 105s 6s/step - loss: 5.9898 - accuracy: 0.5290 - val_loss: 1.7832 - val_accuracy: 0.7374
Epoch 2/50
17/17 [==============================] - 95s 6s/step - loss: 0.8358 - accuracy: 0.8307 - val_loss: 1.2578 - val_accuracy: 0.7529
Epoch 3/50
17/17 [==============================] - 95s 6s/step - loss: 0.2545 - accuracy: 0.9246 - val_loss: 1.0260 - val_accuracy: 0.8152
Epoch 4/50
17/17 [==============================] - 95s 6s/step - loss: 0.0772 - accuracy: 0.9762 - val_loss: 1.1837 - val_accuracy: 0.7879
Epoch 5/50
17/17 [==============================] - 96s 6s/step - loss: 0.1631 - accuracy: 0.9484 - val_loss: 1.4035 - val_accuracy: 0.7802
Epoch 6/50
17/17 [==============================] - 95s 6s/step - loss: 0.1036 - accuracy: 0.9635 - val_loss: 1.3597 - val_accuracy: 0.7879
Epoch 7/50
17/17 [==============================] - 95s 6s/step - loss: 0.0374 - accuracy: 0.9869 - val_loss: 1.0928 - val_accuracy: 0.8113
Epoch 8/50
17/17 [=

In [37]:
# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_ds, test_labels)
# Print the accuracy
print('Accuracy:', accuracy)

35/35 [==============================] - 44s 1s/step - loss: 0.9408 - accuracy: 0.8329
Accuracy: 0.8328791856765747


In [38]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(128, activation='relu')
dense_layer_2 = layers.Dense(64, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')

model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    tf.keras.layers.Dropout(0.5),
    dense_layer_2,
    tf.keras.layers.Dropout(0.5),
    prediction_layer
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [39]:
model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=128, callbacks=[es])

Epoch 1/50
17/17 [==============================] - 106s 6s/step - loss: 4.5470 - accuracy: 0.3460 - val_loss: 1.3184 - val_accuracy: 0.5097
Epoch 2/50
17/17 [==============================] - 98s 6s/step - loss: 1.8465 - accuracy: 0.3202 - val_loss: 1.4432 - val_accuracy: 0.4125
Epoch 3/50
17/17 [==============================] - 99s 6s/step - loss: 1.6676 - accuracy: 0.3689 - val_loss: 1.2279 - val_accuracy: 0.5175
Epoch 4/50
17/17 [==============================] - 106s 6s/step - loss: 1.5318 - accuracy: 0.3873 - val_loss: 1.1379 - val_accuracy: 0.4883
Epoch 5/50
17/17 [==============================] - 99s 6s/step - loss: 1.4492 - accuracy: 0.4068 - val_loss: 0.9768 - val_accuracy: 0.6148
Epoch 6/50
17/17 [==============================] - 98s 6s/step - loss: 1.4144 - accuracy: 0.4175 - val_loss: 0.8935 - val_accuracy: 0.6128
Epoch 7/50
17/17 [==============================] - 98s 6s/step - loss: 1.4508 - accuracy: 0.4550 - val_loss: 0.9372 - val_accuracy: 0.6012
Epoch 8/50
17/17 [

In [40]:
# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_ds, test_labels)
# Print the accuracy
print('Accuracy:', accuracy)

35/35 [==============================] - 44s 1s/step - loss: 0.7872 - accuracy: 0.8047
Accuracy: 0.8047229647636414


## Visualização
Abaixo segue alguns exemplo classificados de maneira incorreta.


In [41]:

#i = 5
#allwrongs = [[],[],[]]
#for X_batch, y_batch in data.val_dataloader():
#    preds=tf.argmax(model(X_batch),axis=1,output_type = tf.int32)
#    wrong=preds!=y_batch
#    X_wrongs,y_wrongs,preds_wrongs=X_batch[wrong],y_batch[wrong],preds[wrong]
#    allwrongs[0].extend(X_wrongs),allwrongs[1].extend(y_wrongs),allwrongs[2].extend(preds_wrongs)
#    if len(X_wrongs) > 1:
#        if randint(1,10) == 2:
#            d2l.show_images(tf.squeeze(X_wrongs), 1, len(X_wrongs),titles=[(int(y),int(pred)) for (y,pred) in zip(y_wrongs[:len(X_wrongs)],preds_wrongs[:len(X_wrongs)])])
#            i-=1
#            if i == 0 : break
#
#d2l.show_images(tf.squeeze(allwrongs[0]), 2, 4,titles=[(int(y),int(pred)) for (y,pred) in zip(allwrongs[1][:8],allwrongs[2][:8])]);plt.show()

## Matriz de Confusão

In [42]:
#num_labels=[f'nº {i}' for i in range(10)]
#
#mini_batch_confusion_matrix=confusion_matrix([],[],labels=[i for i in range(10)])
#
#for X, y in data.val_dataloader():
#    preds=tf.argmax(model(X),axis=1)
#    mini_batch_confusion_matrix += confusion_matrix(y,preds,labels=[i for i in range(10)])
#
#ConfusionMatrixDisplay(confusion_matrix=mini_batch_confusion_matrix,display_labels=num_labels).plot(colorbar=0)
#plt.show()



## Análise
    Observando a matriz de confusão acima, pode-se notar que as confusões mais comuns e suas relativas semelhanças são entre:
- 4 com 9 => topo arredondado
- 7 com 2 => reta relativamente horizontal na parte superior
- 5 com 3 => semi-circulo na parte inferior
- 3 com 8 => não vejo

    Desse erros, conseguimos perceber certos padrões para justificar os erros. Porém a 4ª confusão não faz sentido.  